In [1]:
import pandas as pd
import random
import folium
from geopy.distance import distance

def map_construction_sites(retailers_file, construction_sites_file, output_file):
    retailers_data = pd.read_excel(retailers_file)
    retailer_locations = retailers_data[['Latitude', 'Longitude']].values.tolist()
    retailer_ids = retailers_data['Retailer'].values.tolist()

    construction_data = pd.read_excel(construction_sites_file)
    construction_locations = construction_data[['Latitude', 'Longitude']].values.tolist()
    fence_coordinates = []
    location_ids = []
    colors = []

    radius = 1000

    for i, location in enumerate(retailer_locations):
        latitude, longitude = location
        location_id = f"Location_{i + 1}"
        color = "#{:06x}".format(random.randint(0, 0xFFFFFF))
        location_ids.append(location_id)
        colors.append(color)
        fence_coordinates.append((latitude, longitude))

    map = folium.Map(location=retailer_locations[0], zoom_start=12, tiles='OpenStreetMap')

    # Add retailer geo fences to the map
    for i, fence_coord in enumerate(fence_coordinates):
        location = retailer_locations[i]
        folium.Circle(
            location=fence_coord,
            radius=radius,
            color=colors[i],
            fill=True,
            fill_color=colors[i],
            fill_opacity=0.4
        ).add_to(map)
        folium.Marker(
            location=location,
            tooltip=location_ids[i]
        ).add_to(map)


    for construction_location in construction_locations:
        folium.Rectangle(
            bounds=[(construction_location[0] - 0.001, construction_location[1] - 0.001),
                    (construction_location[0] + 0.001, construction_location[1] + 0.001)],
            color='yellow',
            fill=True,
            fill_color='yellow',
            fill_opacity=0.4
        ).add_to(map)

    construction_retailer_ids = []
    construction_geo_fences = []

    for construction_location in construction_locations:
        within_fences = []
        retailer_ids_within_fences = []

        for i, fence_coord in enumerate(fence_coordinates):
            point_distance = distance(fence_coord, construction_location).meters

            if point_distance <= radius:
                within_fences.append(location_ids[i])
                retailer_ids_within_fences.append(retailer_ids[i])

        if within_fences:
            sorted_retailer_ids = sorted(retailer_ids_within_fences, key=lambda x: construction_retailer_ids.count(x))
            construction_retailer_ids.append(sorted_retailer_ids)
            construction_geo_fences.append(within_fences)
        else:
            construction_retailer_ids.append(['Not_mapped_retailer'])
            construction_geo_fences.append(None)


    retailer_construction_data = pd.DataFrame(columns=['Retailer', 'Construction Sites'])

    for i, retailer_id in enumerate(retailer_ids):
        construction_sites = [construction_data['Construction_id'][j] for j, ids in enumerate(construction_retailer_ids) if retailer_id in ids]
        if not construction_sites:
            construction_sites.append('undefined_construction_site')
        retailer_construction_data.loc[i] = [retailer_id, ', '.join(construction_sites)]

    # Check for undefined retailers
    for i, construction_sites in enumerate(construction_data['Construction_id']):
        if not any(retailer_id in construction_retailer_ids[i] for retailer_id in retailer_ids):
            retailer_construction_data.loc[len(retailer_construction_data)] = ['undefined_retailer', construction_sites]

    with pd.ExcelWriter(output_file) as writer:
        retailer_construction_data.to_excel(writer, sheet_name='Construction_Data', index=False)

    map.save("geo_fence_map.html")

    print('Construction sites mapped to retailers. Output saved in:', output_file)

map_construction_sites('/content/retailers_data.xlsx', '/content/construction_sites.xlsx', '/content/OUTPUT_construction_data.xlsx')

Construction sites mapped to retailers. Output saved in: /content/OUTPUT_construction_data.xlsx
